In [1]:
import pandas as pd
import os

In [2]:
df = pd.read_csv('merged_output.csv')

threshold = 0.98
total_rows = len(df)

for col in df.columns:
    zero_ratio = (df[col] == 0).sum() / total_rows
    if zero_ratio > threshold:
        print(f"{col}: {zero_ratio*100:.2f}% là 0")

Bwd PSH Flags: 100.00% là 0
Fwd URG Flags: 99.97% là 0
Bwd URG Flags: 100.00% là 0
FIN Flag Count: 99.75% là 0
CWE Flag Count: 99.97% là 0
Fwd Avg Bytes/Bulk: 100.00% là 0
Fwd Avg Packets/Bulk: 100.00% là 0
Fwd Avg Bulk Rate: 100.00% là 0
Bwd Avg Bytes/Bulk: 100.00% là 0
Bwd Avg Packets/Bulk: 100.00% là 0
Bwd Avg Bulk Rate: 100.00% là 0


In [ ]:
total_rows = len(df)
#1. Đếm dòng chứa NaN
nan_rows = df.isnull().any(axis=1).sum()
print(f"Số dòng chứa giá trị NaN: {nan_rows}")
#2. Đếm dòng chứa vô cùng
inf_rows = df.applymap(lambda x: str(x).lower() in ["inf", "-inf", "infinity", "-infinity"]).any(axis=1).sum()
inf_numeric_rows = ((df == np.inf) | (df == -np.inf)).any(axis=1).sum()
total_inf_rows = inf_rows + inf_numeric_rows
print(f"Số dòng chứa giá trị vô cùng (inf, -inf): {total_inf_rows}")
#3. Đếm số dòng bị lặp toàn phần
duplicated_rows = df.duplicated()
duplicate_count = duplicated_rows.sum()
print(f"Số dòng bị lặp toàn bộ: {duplicate_count}")
#4. Đếm số dòng lặp theo từng nhãn
if 'Label' in df.columns:
    duplicate_by_label = df[duplicated_rows].groupby('Label').size()
    print("\nSố dòng bị lặp theo từng nhãn (Label):")
    print(duplicate_by_label)
else:
    print("\nKhông tìm thấy cột 'Label' trong dữ liệu.")

Số dòng chứa giá trị NaN: 0
Số dòng chứa giá trị vô cùng (inf, -inf): 0
Số dòng bị lặp toàn bộ: 331272

Số dòng bị lặp theo từng nhãn (Label):
Label
Benign              326591
Brute Force -Web        15
Brute Force -XSS         1
Infilteration         4664
SQL Injection            1
dtype: int64


In [6]:
columns_to_drop = [
    "Bwd PSH Flags",
    "Fwd URG Flags",
    "Bwd URG Flags",
    "FIN Flag Count",
    "CWE Flag Count",
    "Fwd Avg Bytes/Bulk",
    "Fwd Avg Packets/Bulk",
    "Fwd Avg Bulk Rate",
    "Bwd Avg Bytes/Bulk",
    "Bwd Avg Packets/Bulk",
    "Bwd Avg Bulk Rate"
]

# 1. Xóa các cột nếu chúng tồn tại
df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)

# 2. Chuyển kiểu dữ liệu các cột (trừ nhãn 'Label') sang float64, xử lý lỗi chuyển kiểu
for col in df.columns:
    if col != 'Label':
        df[col] = pd.to_numeric(df[col], errors='coerce').astype('float64')
df.dropna(inplace=True)

# 3. Xóa các dòng trùng lặp
df_cleaned = df.drop_duplicates()

# 4. Lưu kết quả ra file mới
df_cleaned.to_csv("cleaned_output.csv", index=False)

print("Đã làm sạch và lưu vào cleaned_output.csv")

Đã làm sạch và lưu vào cleaned_output.csv
